In [1]:
import numpy as np; pi = np.pi
from qutip import * 
from qutip.qip.operations import *
from qutip.qip.circuit import *
from cvqaoa import *

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## RZ

Specify the targeted average gate fidelity

In [3]:
d = 2 # dimension
tau = 2 # gate time
f_bar = 0.99648 # target average gate fidelity
gamma = 2*(d+1)/(d*tau)*(1-f_bar)
T1 = 1/(gamma)
print('T1',T1)

T1 189.39393939394114


Create a qubit processor

In [4]:
## Qubit Processor
qp = QubitProcessor(N=1,T1=T1)
## Pauli matrices
sigma = [sigmax(), sigmay(), sigmaz()]

Calcualte the average gate fidelity

In [5]:
avg_fid = []
## List of angles
arg_list = np.linspace(0,np.pi/2,10)

## Loop over the list of angles and calculate the average gate fidelity
for arg in arg_list:
    ## Create quantum circuit
    qc = QubitCircuit(N=1)
    qc.add_gate("RZ", 0, None, arg)
    
    ## Ideal gate
    U = (-1j*arg/2*sigmaz()).expm()
    
    ## Average Gate Fidelity
    F = 0
    for sigma_k in sigma:
        ## Master equation
        result = qp.run_state(init_state=sigma_k, qc=qc)
        final_state = result.states[-1]
        ## Target state
        target_state = U * sigma_k * U.dag()
        F += (target_state * final_state).tr().real
    avg_fid.append(1/2 + 1/12 * F)

In [7]:
print('Average Gate Fidelity =',round(np.mean(avg_fid),5))

Average Gate Fidelity = 0.99649


## RX

Specify the targeted average gate fidelity

In [26]:
d = 2 # dimension
tau = 10 # gate time
f_bar = 0.99 # target average gate fidelity
gamma = 2*(d+1)/(d*tau)*(1-f_bar)
T1 = 1/(gamma)
print('T1',T1)

T1 333.33333333333303


Create a qubit processor

In [27]:
## Qubit Processor
qp = QubitProcessor(N=1,T1=T1)
## Pauli matrices
sigma = [sigmax(), sigmay(), sigmaz()]

Calcualte the average gate fidelity

In [28]:
avg_fid = []
## List of angles
arg_list = np.linspace(0,np.pi/2,20)

## Loop over the list of angles and calculate the average gate fidelity
for arg in arg_list:
    ## Create quantum circuit
    qc = QubitCircuit(N=1)
    qc.add_gate("RX", 0, None, arg)
    
    ## Ideal gate
    U = (-1j*arg/2*sigmax()).expm()
    
    ## Average Gate Fidelity
    F = 0
    for sigma_k in sigma:
        ## Master equation
        result = qp.run_state(init_state=sigma_k, qc=qc)
        final_state = result.states[-1]
        ## Target state
        target_state = U * sigma_k * U.dag()
        F += (target_state * final_state).tr().real
    avg_fid.append(1/2 + 1/12 * F)

In [29]:
print('Average Gate Fidelity =',round(np.mean(avg_fid),4))

Average Gate Fidelity = 0.9901


## ZZ

Specify the targeted average gate fidelity

In [30]:
d = 4 # dimension
tau = 2 # gate time
f_bar = 0.99 # target average gate fidelity
gamma = (d+1)/(d*tau)*(1-f_bar)
T1 = 1/(gamma)
print('T1',T1)

T1 159.99999999999986


Create a qubit processor

In [31]:
## Qubit Processor
qp = QubitProcessor(N=2,T1=T1)
## Pauli matrices
sigma = [qeye(2), sigmax(), sigmay(), sigmaz()]

In [32]:
def carb(arg_value):
    # control arbitrary phase gate
    zz = tensor(sigmaz(),sigmaz())
    return (-1j*arg_value/2*zz).expm()

Calcualte the average gate fidelity

In [33]:
avg_fid = []
## List of angles
arg_list = np.linspace(0,np.pi/2,20)

## Loop over the list of angles and calculate the average gate fidelity
for arg in arg_list:
    ## Create quantum circuit
    qc = QubitCircuit(N=2)
    qc.user_gates = {"CARB": carb}
    qc.add_gate("CARB", targets = [0,1], arg_value = arg)
    
    ## Ideal gate
    U = (-1j*arg/2*tensor(sigmaz(),sigmaz())).expm()
    
    ## Average Gate Fidelity
    F = 0
    for sigma_k in sigma:
        for sigma_l in sigma:
            sigma_kl = tensor(sigma_k,sigma_l)
            ## Master equation
            result = qp.run_state(init_state=sigma_kl, qc=qc)
            final_state = result.states[-1]
            ## Target state
            target_state = U * sigma_kl * U.dag()
            F += (target_state * final_state).tr().real
    avg_fid.append((F + d**2) / (d**2*(d+1)))

In [34]:
print('Average Gate Fidelity =',round(np.mean(avg_fid),4))

Average Gate Fidelity = 0.9901
